In [36]:
import os
import requests
import xlrd
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [37]:
api_key = os.getenv("KEY")
url = f"https://api.eia.gov/v2/petroleum/pri/spt/data/?frequency=monthly&data[0]=value&facets[product][]=EPCWTI&start=1986-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key}"

6RwPTOz6tfMEjFLYbg4XBiszSSeG5sOu2Tr8NFBf


In [38]:
r = requests.get(url)

In [39]:
json = r.json()

dict

In [51]:
df = pd.DataFrame(json['response']['data'])

In [52]:
df

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2022-12,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",76.44,$/BBL
1,2022-11,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",84.37,$/BBL
2,2022-10,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",87.55,$/BBL
3,2022-09,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",84.26,$/BBL
4,2022-08,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",93.67,$/BBL
...,...,...,...,...,...,...,...,...,...,...,...
439,1986-05,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",15.38,$/BBL
440,1986-04,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",12.84,$/BBL
441,1986-03,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",12.61,$/BBL
442,1986-02,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",15.46,$/BBL
